In [ ]:
import os
from rmgpy.rmg.main import RMG, RMG_Memory, log_conditions

## run an rmg model
### 1. create a new rmg model from an input file

In [ ]:
current_dir = os.path.abspath('')
output_dir = os.path.join(current_dir, 'output')

# # instantiate the profiler
# rmg_profiler = cProfile.Profile()
# stats_file = os.path.join(args.output_directory, 'RMG.profile')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

input_file = 'input.py'

with open(input_file, 'r') as f:
    rmg_input = f.read()

rmg_job = RMG(input_file=input_file, output_directory=output_dir)


### 2. initialize job

In [ ]:
rmg_job.initialize()

# initialize rmg memories object 
rmg_job.rmg_memories = []

# Initialize memory object to track conditions for ranged reactors
rmg_job.rmg_memories.append(RMG_Memory(rmg_job.reaction_systems[0], rmg_job.balance_species))
rmg_job.rmg_memories[0].generate_cond()
log_conditions(rmg_job.rmg_memories, 0)


### 3. expand model

In [ ]:
# React core species to enlarge edge
rmg_job.reaction_model.enlarge(react_edge=True,
                            unimolecular_react=rmg_job.unimolecular_react,
                            bimolecular_react=rmg_job.bimolecular_react,
                            trimolecular_react=rmg_job.trimolecular_react)

For the simple reactor, what we want to do first is identify what we want to parallelize. 
The first target is the core/edge enlargement routine, because it goes through all of the edge species in serial.

the `simulate` routine is inherited from the `base` class for SimpleReactor. Within
it, we can see that the  

In [ ]:
model_settings = rmg_job.model_settings_list[0]
simulator_settings = rmg_job.simulator_settings_list[0]
terminated, resurrected, obj, new_surface_species, new_surface_reactions, t, x = rmg_job.reaction_systems[0].simulate(
    core_species=rmg_job.reaction_model.core.species,
    core_reactions=rmg_job.reaction_model.core.reactions,
    edge_species=rmg_job.reaction_model.edge.species,
    edge_reactions=rmg_job.reaction_model.edge.reactions,
    surface_species=rmg_job.reaction_model.surface.species,
    surface_reactions=rmg_job.reaction_model.surface.reactions,
    pdep_networks=rmg_job.reaction_model.network_list,
    prune=False,
    model_settings=model_settings,
    simulator_settings=simulator_settings,
    conditions=rmg_job.rmg_memories[0].get_cond()
)

In [ ]:
obj

In [ ]:
new_surface_species, new_surface_reactions

In [ ]:
type(rmg_job.reaction_systems[0])